In [1]:
from atena.evaluation.metrics import (
    EvalInstance,
    DisplaysTreeBleuMetric,
    PrecisionMetric,
    get_dataframe_all_eval_metrics
)
from atena.simulation.dataset import (
    Dataset,
    DatasetMeta,
    DatasetName,
    CyberDatasetName,
    FlightsDatasetName,
    SchemaName,
)
from atena.simulation.actions import (
    GroupAction,
    Column,
    AggregationFunction,
    BackAction,
    FilterAction,
    FilterOperator,
)
from atena.simulation.actions_simulator import ActionsSimulator
from atena.simulation.utils import random_action_generator

# Datasets
Our repository contains 8 datasets:

1. 4 datasets in the **cyber** domain that each depicts some network attack:
    * **DATASET1:** ICMP scan on IP range
    * **DATASET2:** Remote code execution attack
    * **DATASET3:** Web-based phishing attack
    * **DATASET4:** TCP port scan
    

2. 4 datasets in the **flights** domain containging data with regard to flight delays:
    * **DATASET1:** AA Flights on Sundays
    * **DATASET2:** Flights departing from BOS
    * **DATASET3:**  From SFO to LAX
    * **DATASET4:** Short, night-time flights


## Loading a Dataset
Loading a dataset is as simple as using the `load_data` method of the `Dataset` class:

In [2]:
dataset_meta = DatasetMeta(SchemaName.CYBER, CyberDatasetName.DATASET1)
dataset = Dataset(dataset_meta)
dataset_df = dataset.load_data()

display(dataset_df.head(5))

,packet_number,captured_length,eth_dst,eth_src,highest_layer,info_line,interface_captured,ip_dst,ip_src,length,number,project_id,sniff_timestamp,tcp_dstport,tcp_srcport,tcp_stream
0,0,66,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,TCP,8888?34374 [ACK] Seq=1 Ack=1 Win=65535 Len=0 T...,NaN,192.168.1.1,192.168.1.122,66,1,1,2013-06-02 10:19:14,34374.0,8888.0,0.0
1,1,74,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=256/1, ttl...",NaN,1.2.3.4,192.168.1.122,74,2,1,2013-06-02 10:19:15,NaN,NaN,NaN
2,2,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=512/2, ttl...",NaN,82.108.24.168,192.168.1.122,75,3,1,2013-06-02 10:19:19,NaN,NaN,NaN
3,3,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=768/3, ttl...",NaN,82.108.25.192,192.168.1.122,75,4,1,2013-06-02 10:19:19,NaN,NaN,NaN
4,4,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=1024/4, tt...",NaN,82.108.6.189,192.168.1.122,75,5,1,2013-06-02 10:19:19,NaN,NaN,NaN


# Analysis Actions
Our system supports the following EDA Actions:
* **Back**
* **Filter** using a filter operator (condition) and a filter term
* **Group** with some aggregation

## Executing Actions Using ActionsSimulator
To execute an action over a dataset we use an object of type `ActionsSimulator` and the method `execute_action` defined in this class. The result of executing an action is of type `ActionExecutionResult`. It contains the property `df` that returns the resulted DataFrame after executing the action. Here is an example:

In [3]:
actions_simulator = ActionsSimulator(dataset=dataset)
action_execution_result = actions_simulator.execute_action(
    GroupAction(grouped_column=Column('highest_layer'), aggregated_column=Column('packet_number'),
                aggregation_function=AggregationFunction.COUNT))
action_execution_result.df

,packet_number
highest_layer,
ARP,4
ICMP,7677
TCP,967


We can now run another action on top of the previous action:

In [4]:
action_execution_result = actions_simulator.execute_action(
    GroupAction(grouped_column=Column('ip_src'), aggregated_column=Column('packet_number'),
                aggregation_function=AggregationFunction.COUNT))
action_execution_result.df.head(5)

packet_number
highest_layer ip_src                       
ICMP          192.168.1.122            7304
              212.134.198.29              2
              217.206.63.73               2
              82.108.1.16                 2
              82.108.1.69                 2

To run a list of actions and get the a list of all the `ActionExecutionResult` objects created due to the execution of those actions, use `run_actions` of the `ActionsSimulator` class, as follows:

In [5]:
actions_lst = [
    GroupAction(grouped_column=Column('highest_layer'), aggregated_column=Column('packet_number'),
                aggregation_function=AggregationFunction.COUNT),
    BackAction(),
    GroupAction(grouped_column=Column('eth_src'), aggregated_column=Column('packet_number'),
                aggregation_function=AggregationFunction.COUNT),
    GroupAction(grouped_column=Column('ip_src'), aggregated_column=Column('packet_number'),
                aggregation_function=AggregationFunction.COUNT),
    FilterAction(filtered_column=Column('info_line'), filter_operator=FilterOperator.CONTAINS,
                 filter_term='Echo (ping) reply'),
]


action_execution_result_lst = actions_simulator.run_actions(actions_lst)

display(action_execution_result_lst[0].df)
display(action_execution_result_lst[1].df.head(5))
display(action_execution_result_lst[2].df)
display(action_execution_result_lst[3].df.head(5))
display(action_execution_result_lst[4].df.head(5))

,packet_number
highest_layer,
ARP,4
ICMP,7677
TCP,967


,packet_number,captured_length,eth_dst,eth_src,highest_layer,info_line,interface_captured,ip_dst,ip_src,length,number,project_id,sniff_timestamp,tcp_dstport,tcp_srcport,tcp_stream
0,0,66,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,TCP,8888?34374 [ACK] Seq=1 Ack=1 Win=65535 Len=0 T...,NaN,192.168.1.1,192.168.1.122,66,1,1,2013-06-02 10:19:14,34374.0,8888.0,0.0
1,1,74,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=256/1, ttl...",NaN,1.2.3.4,192.168.1.122,74,2,1,2013-06-02 10:19:15,NaN,NaN,NaN
2,2,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=512/2, ttl...",NaN,82.108.24.168,192.168.1.122,75,3,1,2013-06-02 10:19:19,NaN,NaN,NaN
3,3,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=768/3, ttl...",NaN,82.108.25.192,192.168.1.122,75,4,1,2013-06-02 10:19:19,NaN,NaN,NaN
4,4,75,00:26:b9:2b:0b:59,00:0c:29:54:bf:79,ICMP,"Echo (ping) request id=0x0200, seq=1024/4, tt...",NaN,82.108.6.189,192.168.1.122,75,5,1,2013-06-02 10:19:19,NaN,NaN,NaN


,packet_number
eth_src,
00:0c:29:54:bf:79,7834
00:26:b9:2b:0b:59,814


packet_number
eth_src           ip_src                       
00:0c:29:54:bf:79 192.168.1.122            7832
00:26:b9:2b:0b:59 212.134.198.29              2
                  217.206.63.73               2
                  82.108.1.16                 2
                  82.108.1.69                 5

packet_number
eth_src           ip_src                      
00:26:b9:2b:0b:59 82.108.1.16                2
                  82.108.1.69                2
                  82.108.10.107              2
                  82.108.10.135              2
                  82.108.10.140              2

# Evaluation
Our goal is to generate a high quality notebook of EDA actions. To determine the quality of a propsed notebook for a specific dataset, we collected for each dataset 5-7 gold standrd references. They can be found in this repository under `atena\evaluation\references` These references were created by expereinced analysts especially for demonstation purposes.

## Evaluation Metrics

To evaluate the quality of the actions we use the following metrics:
1. **Precision**
2. **Tree BLEU** (a variant of the BLEU metric used in NLP that we changed to fit trees). This measure use an additional argument to determine the length of tree sub paths, similar to n-grams in BLEU.
3. **EDA-Sim** - A similarity measure between two trees of displays that uses tree edit distance.

Note that `AbstractMetric` (and all its subclasses) is expected to get a list(!) of objects of type `EvalInstance` and its `compute` method returns the aggregated score of the metric over all instances.

For example, to compute T-BLEU2 of the `actions_lst` we created earlier, we do the following:

In [6]:
eval_instance = EvalInstance(dataset_meta, actions_lst)

# Note that AbstractMetric (and all its subclasses) is expected to
# get a list(!) of objects of type EvalInstance
bleu2 = DisplaysTreeBleuMetric(2, [eval_instance])
bleu2.compute()

0.31140322391459774

To compute also the precision, simply do this:

In [7]:
precision = PrecisionMetric([eval_instance])
precision.compute()

1.0

## Evaluation Example
Here, we generate for each dataset 12 random actions and we give the aggregated (average for non Tree-Blue metrics and corupus-aggregated for Tree Bleu metrics) score of each metric.

### Creating evaluation instances

In [8]:
eval_instances = []

for schema in SchemaName:
    for dataset_name in schema.dataset_names:
        eval_dataset_meta = DatasetMeta(schema, dataset_name=dataset_name)
        eval_instance = EvalInstance(
            eval_dataset_meta,
            actions_lst=[random_action_generator(Dataset(eval_dataset_meta)) for _ in range(12)]
        )
        eval_instances.append(eval_instance)

### Compute all metrics' scores
This can take some time. Go and grab some coffee...

In [ ]:
get_dataframe_all_eval_metrics(eval_instances)